In [1]:
# Imports
from __future__ import print_function

# Keras
import keras

# Other
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd



Using TensorFlow backend.


In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Load Data

In [4]:
from utils.prepare_data import*

ref_q, inputs = load_data(verbose=True)



Reference questions:
 [['We hold ourselves and our team members accountable for results'
  'TEA.2']
 ['The information I need to do my job effectively is readily available'
  'ENA.3']
 ['I know what I need to do to be successful in my role' 'ALI.5']
 ['We are encouraged to be innovative even though some of our initiatives may not succeed'
  'INN.2']]

Labelled data:
 [['I know what my goals are and what I need to do to be successful in my role'
  'ALI.5']
 ['I feel like I can be successful in my role' 'ALI.5']
 ['I know what I need to do to be successful in my role' 'ALI.5']
 ['I understand my role and what is expected of me' 'ALI.5']
 ['I know what is expected of me in my role.' 'ALI.5']]


In [5]:
from utils.prepare_data import *
x_train, y_train, tokenizer, max_length, vocab_size = create_training_vectors(inputs)



Inputs shape:  (297, 20)
Targets shape:  (297, 4)


## Train Network

In [6]:
# Train Model
from utils.model import *
from utils.plots import *
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import os

if not os.path.isfile('model/model'):
    print('Building model...')
    model = build_model(vocab_size, max_length, verbose=False)

    # Callbacks
    checkpointer = ModelCheckpoint('model/model', monitor='val_loss', verbose=0, save_best_only=True, \
                                   save_weights_only=False, mode='auto', period=1)

    print('Training model...')
    history = model.fit(x_train, y_train,
                       batch_size=32, epochs=250,
                       verbose=0,
                       validation_split=0.2,
                       shuffle=True,
                       callbacks=[checkpointer])
    
    plot_loss(history)
    
else:
    print('Loading pretrained model...')
    model = load_model('model/model')
    print('Pretrained model loaded')

Loading pretrained model...
Pretrained model loaded


In [7]:
# Evalute Model

score = model.evaluate(x_train, y_train, verbose=0)
print('    Overall Loss: %.4f' % score[0])
print('Overall Accuracy: %.2f%%' % (score[1]*100))

    Overall Loss: 0.0276
Overall Accuracy: 98.99%


## Test Model

In [17]:
from utils.prepare_data import *

x_test, test_inputs = prepare_test_data(tokenizer, max_length, verbose=True)



Test data:
 ['"My team is enthusiatic about the work we do"'
 '"I understand what I need to do to move up at StarWars"'
 '"I understand clearly what I need to do to be successful in my current role"'
 '"Team members are collectively accountable for achievements"'
 '"Is available when I really need them"'
 '"We are encouraged to be innovative even though some of our initiatives may not succeed"'
 '"I know what is expected of me to be successful in my role"'
 '"The information I need to do my job effectively is readily available"'
 '"We hold ourselves and our team members accountable for results"'
 '"The information I need to do my job effectively is readily available"']


In [18]:
from utils.model import *

test_probs = predict_with_model(x_test, model)

In [19]:
from utils.results import *
display_results(test_probs, test_inputs, ref_q)




Question #1: "My team is enthusiatic about the work we do"
Primary Reference Question: We hold ourselves and our team members accountable for results
Primary Reference Question Code: TEA.2
Relevance: 99.549%
Secondary Reference Question: The information I need to do my job effectively is readily available
Secondary Reference Question Code: ENA.3
Relevance: 0.340%


Question #2: "I understand what I need to do to move up at StarWars"
Primary Reference Question: I know what I need to do to be successful in my role
Primary Reference Question Code: ALI.5
Relevance: 99.538%
Secondary Reference Question: The information I need to do my job effectively is readily available
Secondary Reference Question Code: ENA.3
Relevance: 0.371%


Question #3: "I understand clearly what I need to do to be successful in my current role"
Primary Reference Question: I know what I need to do to be successful in my role
Primary Reference Question Code: ALI.5
Relevance: 99.653%
Secondary Reference Question: Th